# 1. Importar las bibliotecas a utilizar:

In [7]:
import cobra  
%matplotlib inline
from cobra import Reaction, Metabolite
import gurobipy
from cobra.io import read_sbml_model
from cobra.io import write_sbml_model

# 2. Cargar los Modelos

En este caso los modelos se obtuvieron del repositorio Bigg (http://bigg.ucsd.edu) en formato json. Conla función display() puedo obtener un resumen del contenido de los modelos

In [8]:
modelCore = cobra.io.load_json_model('../models/e_coli_core.json')
display(modelCore)
modeliML = cobra.io.load_json_model('../models/iML1515.json')
display(modeliML)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-05


Name,e_coli_core
Memory address,0x07faaeb961d30
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


Name,iML1515
Memory address,0x07faaf298c370
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


Se crea una lista de diccionarios que contienen la información y estequiometría de las reacciones:

In [9]:
metReactions = [
    {
        'id':'ACLS',
        'Name': 'Acetolactate synthase',
        'Metabolites': {
            'h_c':-1,
            'pyr_c': -2,
            'alac__S_c':1,
            'co2_c': 1
        },
        'gene_reaction_rule':'(ilvG or ilvH)'
    },
    {
        'id':'ACLDC',
        'Name': 'Acetolactate decarboxylase',
        'Metabolites': {
            'alac__S_c':-1,
            'h_c': -1,
            'actn__R_c':1,
            'co2_c': 1
        },
        'gene_reaction_rule':'(budA)'
        
    }, 
    {
        'id':'BTD_RR',
        'Name': 'R R butanediol dehydrogenase',
        'Metabolites': {
            'btd_RR_c':1,
            'nad_c': 1,
            'actn__R_c':-1,
            'h_c': -1, 
            'nadh_c':-1
        },
        'gene_reaction_rule':'(adh or bdhA)'
    }, 
    {
        'id':'BTDt_RR', 
        'Name': 'R R  butanediol transport',
        'Metabolites': {
            'btd_RR_c':-1,
            'btd_RR_e': 1
        },
        'gene_reaction_rule':''
    },
    {
        'id':'EX_btd_RR_e', 
        'Name': 'R R  2 3 Butanediol exchange',
        'Metabolites': {
            'btd_RR_e':-1
        },
        'gene_reaction_rule':''
        
    }
]

Se define una función para revisar si las reacciones existen o no en el modelo:

In [10]:
def check_reactions(model, metReactions):
    rxExist = []
    rxNexist = []
    for rx in metReactions:
        try: 
            model.reactions.get_by_id(rx['id'])
            rxExist.append(rx)
        except KeyError as e: 
            rxNexist.append(rx)

    print('Las reacciones que están:', rxExist)
    print('Las reacciones que no están:', rxNexist)
    
    return rxExist, rxNexist

Almaceno las reacciones que si existen y las que no existen en listas diferentes:


In [ ]:
rxExistCore, rxNexistCore = check_reactions(modelCore,metReactions)
rxExistiML, rxNexistiML = check_reactions(modeliML,metReactions)

Ahora defino una lista con diccionarios con las definiciones de los metabolitos que deben ser parte del modelo:

In [ ]:
metMetabolites = [
    {
        'id': 'h_c',
        'formula': 'H',
        'name': 'H+',
        'compartment': 'c'
    },
    {
        'id': 'pyr_c',
        'formula': 'C3H3O3',
        'name': 'Pyruvate',
        'compartment': 'c'
    },
    {
        'id': 'alac__S_c',
        'formula': 'C5H8O4',
        'name': '(S)-2-Acetolactate',
        'compartment': 'c'
    },
    {
        'id': 'co2_c',
        'formula': 'CO2',
        'name': 'CO2 CO2',
        'compartment': 'c'
    },
    {
        'id': 'actn__R_c',
        'formula': 'C4H8O2',
        'name': 'R  Acetoin',
        'compartment': 'c'
    },
    {
        'id': 'btd_RR_c',
        'formula': 'C4H10O2',
        'name': 'R R  2 3 Butanediol',
        'compartment': 'c'
    },
    {
        'id': 'nadh_c',
        'formula': 'C21H27N7O14P2',
        'name': 'Nicotinamide adenine dinucleotide - reduced',
        'compartment': 'c'
    },
    {
        'id': 'nad_c',
        'formula': 'C21H26N7O14P2',
        'name': 'Nicotinamide adenine dinucleotide',
        'compartment': 'c'
    },
    {
        'id': 'btd_RR_e',
        'formula': 'C4H10O2',
        'name': 'R R  2 3 Butanediol',
        'compartment': 'e'
    }
]

De la misma forma que con las reacciones, creo una función que revisa que metabolitos o no existen en el modelo y luego almaceno esa información sus listas respectivas

In [ ]:
def check_metabolites(model, metMetabolites):
    metExist = []
    metNexist = []

    for met in metMetabolites:
        try: 
            model.metabolites.get_by_id(met['id'])
            metExist.append(met)
        except KeyError as e: 
            metNexist.append(met)

    print('Los metabolitos que están:', metExist)
    print('Los metabolitos que no están:', metNexist) 
    return metExist, metNexist

In [ ]:
metExistCore, metNexistCore = check_metabolites(modelCore, metMetabolites)
metExistiML, metNexistiML = check_metabolites(modeliML, metMetabolites)

Defino la función que me permite agregar los metabolitos al modelo. Según la biblioteca Cobrapy se deben agregar primero los metabolitos faltantes y luego las reaccione, ya que en base a los metabolitos se construye la estequiometría de las reacciones

In [ ]:
def metabolites_to_add(metNexist):
    met_to_add = []
    for met in metNexist:
        m = Metabolite(id=met['id'], formula=met['formula'], name=met['name'], compartment=met['compartment'])
        met_to_add.append(m)
    print(met_to_add)
    return met_to_add

In [ ]:
met_to_add_core = metabolites_to_add(metNexistCore)
met_to_add_iML = metabolites_to_add(metNexistiML)

Luego defino una función para agregar las reacciones:

In [ ]:
def reactions_to_add(model, rxNexist):
    rxs_to_add = []
    for rx in rxNexist:
        x = Reaction(id=rx['id'], name=rx['Name'], lower_bound=-1000.0)
        if(len(rx['gene_reaction_rule'])>0):
            print('esto es:', rx['gene_reaction_rule'])
            x.gene_reaction_rule = rx['gene_reaction_rule']
        rxs_to_add.append(x)
        model.add_reactions([x])
        x.add_metabolites(rx['Metabolites'])
    print('reactions added: ',rxs_to_add)
    print('\n')

In [ ]:
reactions_to_add(modelCore, rxNexistCore)
reactions_to_add(modeliML, rxNexistiML)

Se pueden definir los límites mínimos y máximos de las reacciones. En este caso, defino que los límites mínimos de las reacciones que forman parte de la síntesis del compuesto de interés sean 0 porque no quiero que sean reversibles

In [ ]:
modelCore.reactions.EX_btd_RR_e.lower_bound = 0
modelCore.reactions.ACLS.lower_bound = 0
modelCore.reactions.ACLDC.lower_bound = 0
modelCore.reactions.BTD_RR.lower_bound = 0
modelCore.reactions.BTDt_RR.lower_bound = 0

modeliML.reactions.EX_btd_RR_e.lower_bound = 0
modeliML.reactions.ACLS.lower_bound = 0
modeliML.reactions.ACLDC.lower_bound = 0
modeliML.reactions.BTD_RR.lower_bound = 0
modeliML.reactions.BTDt_RR.lower_bound = 0

Creo una función para definir el medio bajo el cual se desarrollan los modelos. En este caso para definir medios aeróbicos y anaeróbicos

In [ ]:
def modify_medium(model,met, value):
    modelA = model.copy()
    medium = model.medium
    medium[met] = value
    modelA.medium = medium
    return modelA

In [ ]:
modeliMLA = modify_medium(modeliML, 'EX_o2_e', 0)
modelCoreA = modify_medium(modelCore, 'EX_o2_e', 0)

Finalmente puedo guardar y exportar los modelos en formato xml (o también json) para reutilizar en el futuro

In [ ]:
write_sbml_model(modelCore, "models/modelCore.xml")
write_sbml_model(modelCoreA, "models/modelCoreA.xml")
write_sbml_model(modeliML, "models/modeliML.xml")
write_sbml_model(modeliMLA, "models/modeliMLA.xml")

In [ ]:
display(modelCore)
display(modelCoreA)
display(modeliML)
display(modeliMLA)